## Здравствуйте, меня зовут *Алексей Медведев*, вот моё Задание №3 по курсу ПСАД-2019!

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from collections import defaultdict
from matplotlib import pyplot as plt

import seaborn as sns
sns.set()

np.random.seed(228)

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

### Задача 1  

Задана обучающая выборка $\widetilde{S}$, включающая значения переменных $X$ и $Y$.

In [ ]:
from scipy.stats import linregress

def draw_linear(x, y, x_name='X', y_name='Y'):
    "Scatter x-y points and draw a LR line."
    slope, intercept, r, _, _ = linregress(x, y)
    x_max, x_min = x.max(), x.min()
    x_range = x_max - x_min
    x_lims = np.array([x_min - 0.1 * x_range, x_max + 0.1 * x_range])
    plt.xlabel(x_name)
    plt.ylabel(y_name)
    plt.plot(x, y, 'g+', label='Objects')
    plt.plot(x_lims, intercept + slope * x_lims, 'r--', label=r"LR line with $\rho \approx {:.3f}$".format(r))
    plt.legend()
    plt.show()

In [ ]:
S = np.load("data1.npy")
X, Y = S
draw_linear(X, Y)

Оцените значимость коэффициента корреляции с помощью...

Критерия Стьюдента

In [ ]:
def student_significance(X, Y):
#    a = X - X.mean()
#    b = Y-Y.mean()
#    corr = a.mean()*b.mean()/(((a**2).mean()**0.5)*((b**2).mean()**0.5))
    r = np.corrcoef(X,Y)[0,1]
    t = r*(X.shape[0])**0.5/(1-r**2)**0.5
    significance = (1-sp.stats.t.cdf(t, df = X.shape[0] - 2))
    
    return significance

Перестановочного теста

In [ ]:
def permutation_test_significance(X, Y, n_permutations=10000):
    B = 10000
    n = Y.shape[0]
    r = np.corrcoef(X,Y)[0,1]
    thresholds = np.array([np.corrcoef(X,Y[np.random.permutation(n)])[0,1] for i in range(B)])
    return (thresholds > r).mean()

Сравните результаты и сделайте выводы.

In [ ]:
print("Student significance: ", round(student_significance(X, Y), 5))
print("PT significance: ", round(permutation_test_significance(X, Y), 5))

**Выводы:** Результаты получились довольно схожими, не смотря на то что методы работают совершенно по-разному. Значит полученная оценка достаточно точна. Малость полученного p-значения значит, что гипотеза об отсутствии корреляции скорее всего будет отвергнута

### Задача 2


Сравните две группы $S_1$ и $S_2$ по переменным $X_1, \dots, X_{10}$ с использованием теста...

In [ ]:
S1 = np.load("data2_1.npy")
S2 = np.load("data2_2.npy")

Манна-Уиттни

In [ ]:
def mw_test(X_from_S1, X_from_S2):
    data = np.append(X_from_S1, X_from_S2)
    n = data.shape[0]
    m = X_from_S1.shape[0]
    ranks = sp.stats.rankdata(data)
    u = lambda r,n:r.sum() - n*(n+1)/2
    t = min(u(ranks[:m],m), u(ranks[m:],n-m))
    significance = (1-sp.stats.norm.cdf(np.abs((t - m*(n-m)/2)/(m*(n+1)*(n-m)/12)**0.5)))
    return significance

Колмогорова-Смирнова

In [ ]:
def ks_test(X_from_S1, X_from_S2):
    data = np.append(X_from_S1, X_from_S2)
    f = lambda data, x: (x >= data).mean()
    t = np.max([np.abs(f(X_from_S1,i) - f(X_from_S2,i)) for i in np.unique(data)])
    n = X_from_S1.shape[0]
    m = X_from_S2.shape[0]
    significance = sp.special.kolmogorov(t*(n*m/(n+m))**0.5)
    return significance

In [ ]:
res = defaultdict(list)

for i, (X_from_S1, X_from_S2) in enumerate(zip(S1, S2)):
    res["X"].append(i+1)
    res["Mann–Whitney"].append(mw_test(X_from_S1, X_from_S2))
    res["Kolmogorov–Smirnov"].append(ks_test(X_from_S1, X_from_S2))
    #res["Mann–Whitney-True"].append(sp.stats.mannwhitneyu(X_from_S1, X_from_S2).pvalue)
    
pd.DataFrame(res)

Выясните, какие переменные являются значимыми на уровне $\alpha$ с учётом коррекции...

In [ ]:
alpha = 0.05

Бонферрони

In [ ]:
def bonferroni_corr(p_vals, alpha=alpha):
    "Must return bool (!) array: reject/accept after correction"
    p_vals_ = np.array(p_vals)
    m = p_vals_.shape[0]
    return  p_vals_*m <= alpha

Бонферрони-Холма

In [ ]:
def bh_corr(p_vals, alpha=alpha):
    "Must return bool (!) array: reject/accept after correction"
    p_vals_ = np.array(p_vals)
    idx = np.argsort(p_vals_)
    inv_idx = np.argsort(idx)
    m = p_vals_.shape[0]
    thresh = np.argmax(np.arange(m, 0, -1)*p_vals_[idx] > alpha)
    res = np.ones(m, dtype=bool)
    res[:thresh] = 0
    return np.logical_not(res[inv_idx])

Сравните результаты и сделайте выводы.

In [ ]:
def simple(p_vals, alpha=alpha):
    p_vals_ = np.array(p_vals)
    return p_vals_ <= alpha

In [ ]:
res_corr = defaultdict(list)
res_corr["X"] = res["X"]
res_corr["MW rejected w/ Bonferroni"] = bonferroni_corr(res["Mann–Whitney"])
res_corr["MW rejected w/ BH"] = bh_corr(res["Mann–Whitney"])
res_corr["MW rejected w/ "] = simple(res["Mann–Whitney"])
res_corr["KS rejected w/ Bonferroni"] = bonferroni_corr(res["Kolmogorov–Smirnov"])
res_corr["KS rejected w/ BH"] = bh_corr(res["Kolmogorov–Smirnov"])
res_corr["KS rejected w/"] = simple(res["Kolmogorov–Smirnov"])
pd.DataFrame(res_corr)

*Дополнительная информация:* на самом деле лишь переменные $X_1$, $X_2$ и $X_3$ имеют одинаковое распределение.

**Выводы:** В данных критерях $H_0$ - случайные величины пришли из одного распределения. Зная это можно увидеть, что критерий колмогорова-смирнова в целом дает более близкий к настоящему ответ на вопрос о распределении случайных величин $X_1 \dots X_{10}$. Также можно заметить что использование поправки бонферони увеличивает шанс получения ложноположительных гипотез.

### Задача 3

Найдите линейный временной тренд в предоставленном временном ряду $X_t$.

In [ ]:
def plot_ts(X_t, title):
    plt.plot(X_t)
    plt.title(title)
    plt.xlabel(r"$t$")
    plt.ylabel(r"$X_t$")
    plt.show()

In [ ]:
X_t = np.load("data3.npy")
plot_ts(X_t, "Original time series")

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
LinearRegression().get_params()

In [ ]:
def linear_trend(X_t):
    "Must return floats a, b: LinTrend(X_t) = a * t + b"
#    gap = 10
#    x = np.array([X_t[i:i+gap] for i in range(0,X_t.shape[0]-gap)])
#    y = X_t[gap:]
#    print(x.shape, y.shape)
#    ar = LinearRegression().fit(x, y)
#    a = ar.coef_
#    b = ar.intercept_
    x = np.arange(X_t.shape[0])
    y = X_t
    a,b,_,_,_ = linregress(x,y)
    return a,b

In [ ]:
print("Linear trend: X_t ~ a * t + b")
a, b = linear_trend(X_t)
print("a = ", round(a, 3))
print("b = ", round(b, 3))

In [ ]:
t = np.arange(len(X_t))
plt.plot(t, b + a * t, 'r--', label=r"Linear trend of $X_t$")
plt.legend()
plot_ts(X_t, "Now w/ linear trend")

После вычитания тренда проведите проверку на стохастическую нестационарность с помощью теста Дикки-Фуллера.

In [ ]:
X_t_detrended =  X_t - a * np.arange(X_t.shape[0]) - b
plot_ts(X_t_detrended, "After detrending")

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.adfvalues import mackinnonp
from statsmodels.regression.linear_model import OLS

In [ ]:
def dfuller(X_t_detrended):
    "Must return p-value"
    gap = 1
    x = np.array([X_t_detrended[i:i+gap] for i in range(0,X_t_detrended.shape[0]-gap)])
    y = X_t_detrended[gap:]
    res = OLS(y,x).fit()
    print(res.tvalues[0])
    #res = OLS(X_t_detrended[1:], X_t_detrended[:-1]).fit()
    return mackinnonp(res.tvalues[0])

In [ ]:
print("Detrended X_t is nonstationary with {0} probability.".format(dfuller(X_t_detrended)))

Не забудьте сделать выводы!

**Выводы:** Если убрать тренд, то можно почти сразу заметить, что процесс не является стохастическим. Т.к присутствуют ярко выраженные 'вершины', то распределение проекции процесса начинает зависить от ее координаты.ритерий Дикки-Фуллера дает достаточно большую вероятность того, что процесс не является стохастическим.

In [ ]:
N = int(1e8)

In [ ]:
noise = np.random.normal(0,1,size=(N))

In [ ]:
plt.plot(np.cumsum(noise))

In [ ]:
q = 990
mu = 0

In [ ]:
coef = np.random.normal(size=(q))

In [ ]:
process = np.array([coef.dot(noise[i:i+q]) for i in range(0,N-q)])

In [ ]:
plt.plot(np.arange(N-q), process + mu)